In [1]:
import csv
import base64
import time
import fnmatch
import urllib
from datetime import datetime, timedelta
from github import Github, StatsContributor
from awesomecure import awesome2py
from urllib.parse import urlparse 
from os import getenv, path
from dotenv import load_dotenv
from termcolor import colored

In [2]:
def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timeformat = '{:02d}:{:02d}'.format(mins, secs)
        print(timeformat, end='\r')
        time.sleep(1)
        t -= 1
    print('\n\n\n\n\n')

In [3]:
load_dotenv()
g = Github(getenv("GITHUB"))

In [4]:
awesome_url = "https://github.com/protontypes/open-sustainable-technology"
awesome_path = urlparse(awesome_url).path.strip("/")
filename = "README.md"

awesome_repo = g.get_repo(awesome_path)
awesome_content_encoded = awesome_repo.get_contents(urllib.parse.quote(filename)).content
awesome_content = base64.b64decode(awesome_content_encoded)

filehandle = open('.awesome.md', 'w')
filehandle.write(awesome_content.decode("utf-8"))
filehandle.close()
repo_dict = awesome2py.AwesomeList(".awesome.md")

print(repo_dict)

Photovoltaics and Solar Energy
 - pvlib-python - A set of documented functions for simulating the performance of photovoltaic energy systems. [https://github.com/pvlib/pvlib-python] - pvfactors - Open source view-factor model for diffuse shading and bifacial PV modeling. [https://github.com/SunPower/pvfactors] - gsee - Global Solar Energy Estimator. [https://github.com/renewables-ninja/gsee] - PVMismatch - An explicit Python PV system IV & PV curve trace calculator which can also calculate mismatch. [https://github.com/SunPower/PVMismatch] - rdtools - An open source library to support reproducible technical analysis of time series data from photovoltaic energy systems. [https://github.com/NREL/rdtools] - Machine-Learning-for-Solar-Energy-Prediction - Predict the power production of a solar panel farm from weather measurements using machine learning. [https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction] - elpv-dataset - A dataset of functional and defective solar c

 - Appropedia - Shares knowledge to build rich, sustainable lives. [https://www.appropedia.org/Welcome_to_Appropedia] - Awesome Clean Tech - Clean technology is any process, product, or service that reduces negative environmental impacts through significant energy efficiency improvements, the sustainable use of resources, or environmental protection activities. [https://github.com/nglgzz/awesome-clean-tech] - Awesome Open Climate Science - Awesome Open Atmospheric, Ocean, and Climate Science. [https://github.com/pangeo-data/awesome-open-climate-science] - Awesome Earth - "What can I do about the climate crisis?" Here are 326 things you can do. [https://github.com/philsturgeon/awesome-earth] - Awesome Open Geoscience - Curated from repositories that make our lives as geoscientists, hackers and data wranglers easier or just more awesome. [https://github.com/softwareunderground/awesome-open-geoscience] - Datasets APIs and open source projects related to Climate Change - A curated list of 

In [5]:
csv_github_projects = open('./csv/github_projects.csv', 'w', newline='') 
csv_github_fieldnames = ['project_name','oneliner','project_namespace','project_url', 'topics', 'rubric', 'last_update', 'stargazers_count','project_active_checkbox','dominating_languages', 'organization', 'languages', 'homepage', 'created' , 'license','total_commits_last_year','total_number_of_commits','last_issue_closed', 'open_issues', 'closed_pullrequests', 'closed_issues', 'open_pullrequests','last_released_date','last_release_tag_name','community_development_distribution_score', 'good_first_issue','contributors', 'organization_avatar','accepts_donations', 'donation_platforms','code_of_conduct','contribution_guide','asset_downloads' ]
writer_github_projects = csv.DictWriter(csv_github_projects, fieldnames=csv_github_fieldnames)
writer_github_projects.writeheader()
                                    
csv_github_organizations = open('./csv/github_organizations.csv', 'w', newline='') 
csv_github_organizations_fieldnames =  ['organization_name','organization_avatar','organization_public_repos','organization_location','organization_description','organization_public_members','organization_repos_count','total_organization_stars']
writer_github_organizations = csv.DictWriter(csv_github_organizations, fieldnames=csv_github_organizations_fieldnames)
writer_github_organizations.writeheader()
organization_list = []

csv_gitlab_projects = open('./csv/gitlab_projects.csv', 'w', newline='') 
csv_gitlab_fieldnames = ['project_name','oneliner','project_namespace','project_url', 'rubric']
writer_gitlab_projects = csv.DictWriter(csv_gitlab_projects, fieldnames=csv_gitlab_fieldnames)
writer_gitlab_projects.writeheader()

csv_custom_platform_projects = open('./csv/custom_platform_projects.csv', 'w', newline='') 
csv_custom_platform_fieldnames = ['project_name','oneliner','project_namespace','project_url', 'rubric']
writer_custom_platform_projects = csv.DictWriter(csv_custom_platform_projects, fieldnames=csv_custom_platform_fieldnames)
writer_custom_platform_projects.writeheader()


for r in repo_dict.rubrics:        
    for entry in r.entries: 
        print("------------------------")
        print("Processing %s" % entry.name)
        print("URL: %s" % entry.url)
        if urlparse(entry.url).netloc == 'github.com':
            print("%s is a GitHub project" % entry.name)
            try:

                remaining,limit  = g.rate_limiting
                resettime = g.rate_limiting_resettime
                min_requests = 200
                if(remaining<min_requests):
                    print("------------------------")
                    print("Waiting for more available GitHub requests:")
                    current_time = datetime.now().timestamp() 
                    countdown((int(resettime)-int(current_time)+2))

                print("GitHub Requests | Limit: {}, Remaining: {}".format(limit, remaining))
                # Gather project information from GitHub
                # https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html
                repo_path = urlparse(entry.url).path.strip("/")
                user, project_name  = path.split(repo_path)
                repo = g.get_repo(repo_path)
                contents_root = repo.get_contents("")
                releases = repo.get_releases()
                commits = repo.get_commits()
                
                # Check if the project is still active 
                closed_issues = repo.get_issues(state='closed')
                open_issues = repo.get_issues(state='open')
                
                closed_pullrequests = repo.get_pulls(state='closed')
                open_pullrequests = repo.get_pulls(state='open')

                ## preprocessing
                if closed_issues.totalCount > 0:
                    last_issue_closed = closed_issues[0].updated_at
                    
                total_commits_last_year = 0
                activity = repo.get_stats_commit_activity()
                for event in activity:
                    total_commits_last_year = total_commits_last_year + event.total

                last_issue_closed_timedelta = datetime.utcnow() - last_issue_closed
                
                             # Do not process inactive projects
                if last_issue_closed_timedelta > timedelta(days=365) and total_commits_last_year == 0:
                    print("%s is an inactive project" % entry.name)
                    project_active_checkbox = "false"
                else: 
                    print("%s is an active project" % entry.name)
                    project_active_checkbox = "true"
                
                
                try: 
                    license = repo.get_license()
                    if license.license.spdx_id == 'NOASSERTION':
                        print("Custom license found")
                        license_name = "CUSTOM"
                    else:
                        license_name = license.license.spdx_id
                except:
                    print(colored("No license information found","red"))
                    license_name == "UNDEFINED"
                    

                labels = (",".join([entry.name for entry in repo.get_labels()])) 
                topics = (",".join(repo.get_topics())) 

                languages_states = repo.get_languages()
                programming_languages = (",".join(languages_states.keys()))
                dominating_languages = list(languages_states.keys())[0]

                try:
                    dotfolder_file = repo.get_contents(".github")
                    for file in dotfolder_file:                       
                        if file.path==".github/FUNDING.yml":
                            print("Funding found")
                            donation_platforms = base64.b64decode(file.content)
                            accepts_donations = "true"
                        else:
                            donation_platforms = None
                            accepts_donations = "false"
                except:
                    print(colored("No funding information found","red"))
                    donation_platforms = None
                    accepts_donations = "false"
                
                try:
                    code_of_conduct = "false"
                    contribution_guide = "false"
                    for file_content in contents_root:
                        if file_content.path.lower().find("code_of_conduct") != -1:
                            print("Code of conduct found")
                            code_of_conduct = "true"
                        if file_content.path.lower().find("contributing") != -1:
                            print("Contribution guide found")
                            contribution_guide = "true" 
                            
                    for file_content in dotfolder_file:
                        if file_content.path.lower().find("code_of_conduct") != -1:
                            print("Code of conduct found")
                            code_of_conduct = "true"
                        if file_content.path.lower().find("contributing") != -1:
                            print("Contribution guide found")
                            contribution_guide = "true"
                
                except Exception as e:
                    print(e)
                    

                contributors = repo.get_stats_contributors()
                contributor_activity = {}
                commits_total = 0
                for individuum in contributors:
                    contributor_activity[individuum.author.login] = individuum.total
                    commits_total = commits_total + individuum.total

                sorted_contributor = dict(sorted(contributor_activity.items(), key=lambda item: item[1]))
                weighted_contribution = {k: v / commits_total for k, v in sorted_contributor.items()}
                
                # Create a very simple community health score that shows how much the project is focused on one developer
                community_development_distribution_score = 1 - max(weighted_contribution.values())

                try:   
                    last_released_date = releases[0].published_at.strftime("%Y/%m/%d, %H:%M:%S")
                    last_release_tag_name = releases[0].tag_name
                    asset_downloads = 0
                    for release in releases:
                        assets = release.get_assets()
                        for asset in assets:
                            asset_downloads = asset_downloads + asset.download_count
                    
                except Exception as e:
                    print(colored("No Release found","red"))
                    last_released_date = ""
                    last_release_tag_name = ""
                    print(e)
                    
                total_number_of_commits = commits.totalCount

                 # Gathering organization data 
                if repo.organization is None: 
                    organization_name = None

                elif repo.organization.login not in organization_list: 
                    organization_name = repo.organization.login
                    organization_avatar = repo.organization.avatar_url
                    organization_public_repos = repo.organization.public_repos
                    organization_location = repo.organization.location
                    organization_description = repo.organization.description
                    organization_repos = repo.organization.get_repos()
                    organization_public_members_count = 0
                    
                    
                    total_organization_stars = 0
                    for k in organization_repos:
                        total_organization_stars = total_organization_stars + k.stargazers_count

                    organization_data = {  'organization_name': organization_name,
                                           'organization_avatar': organization_avatar,
                                           'organization_public_repos': organization_public_repos,
                                           'organization_location': organization_location,
                                           'organization_description': organization_description,
                                           'organization_public_members': organization_public_members_count,
                                           'organization_repos_count': organization_repos.totalCount,
                                           'total_organization_stars': total_organization_stars,           
                                         }

                    organization_list.append(organization_name)
                    print("Organization Data:")
                    print(organization_data)
                    writer_github_organizations.writerow(organization_data)
                

                else:
                    organization_name = ''
                    organization_avatar = ''
                
                
                project_data = {  'project_name': entry.name,
                                  'project_namespace': user,
                                  'project_url': repo.clone_url,
                                  'rubric':r.key,
                                  'oneliner':entry.text[2:],
                                  'topics':topics,
                                  'organization':organization_name,
                                  'created':repo.created_at.strftime("%Y/%m/%d, %H:%M:%S"),
                                  'last_update':repo.updated_at.strftime("%Y/%m/%d, %H:%M:%S"),
                                  'total_commits_last_year':total_commits_last_year,
                                  'project_active_checkbox':project_active_checkbox,
                                  'last_issue_closed':last_issue_closed.strftime("%Y/%m/%d, %H:%M:%S"),
                                  'last_released_date':last_released_date,
                                  'last_release_tag_name':last_release_tag_name,
                                  'total_number_of_commits':total_number_of_commits,
                                  'community_development_distribution_score':community_development_distribution_score,
                                  'stargazers_count':repo.stargazers_count,
                                  'asset_downloads':asset_downloads,
                                  'dominating_languages':dominating_languages,
                                  'languages':programming_languages,
                                  'homepage':repo.homepage,
                                  'closed_issues':closed_issues.totalCount,
                                  'open_issues':open_issues.totalCount,
                                  'closed_pullrequests':closed_pullrequests.totalCount,
                                  'open_pullrequests': open_pullrequests.totalCount,
                                  'good_first_issue':repo.get_issues(state='open',labels=['good first issue']).totalCount,
                                  'license':license_name,
                                  'contributors':repo.get_contributors().totalCount,
                                  'accepts_donations':accepts_donations,
                                  'donation_platforms': donation_platforms,
                                  'code_of_conduct': code_of_conduct,
                                  'contribution_guide': contribution_guide
                                }

                print("Project Data:")
                print(project_data)
                writer_github_projects.writerow(project_data)

            except Exception as e:
                print(colored(e,"red"))
                
        elif urlparse(entry.url).netloc == 'gitlab.com':
            print("%s is a Gitlab project" % entry.name)   
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name  = path.split(repo_path)
            
            project_data = {  'project_name': entry.name,
                              'project_namespace': user,
                              'project_url': entry.url,
                              'rubric':r.key,
                              'oneliner':entry.text[2:],
                           }
            print("Project Data:")
            print(project_data)
            writer_gitlab_projects.writerow(project_data)
            
        else:
            print("%s is hosted on custom platform" % entry.name)   
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name  = path.split(repo_path)
            
            project_data = {  'project_name': entry.name,
                              'project_namespace': user,
                              'project_url': entry.url,
                              'rubric':r.key,
                              'oneliner':entry.text[2:],
                           }
            
            print("Project Data:")
            print(project_data)
            writer_custom_platform_projects.writerow(project_data)

csv_github_projects.close()
csv_github_organizations.close()
csv_gitlab_projects.close()
csv_custom_platform_projects.close()

------------------------
Processing pvlib-python
URL: https://github.com/pvlib/pvlib-python
pvlib-python is a GitHub project
GitHub Requests | Limit: 5000, Remaining: 4484
pvlib-python is an active project
Code of conduct found
Contribution guide found
Organization Data:
{'organization_name': 'pvlib', 'organization_avatar': 'https://avatars.githubusercontent.com/u/11037261?v=4', 'organization_public_repos': 4, 'organization_location': None, 'organization_description': None, 'organization_public_members': 0, 'organization_repos_count': 4, 'total_organization_stars': 639}
Project Data:
{'project_name': 'pvlib-python', 'project_namespace': 'pvlib', 'project_url': 'https://github.com/pvlib/pvlib-python.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'A set of documented functions for simulating the performance of photovoltaic energy systems.', 'topics': 'solar-energy,python,renewable-energy,renewables,photovoltaic', 'organization': 'pvlib', 'created': '2015/02/17, 00:21:33', 

{'project_name': 'rdtools', 'project_namespace': 'NREL', 'project_url': 'https://github.com/NREL/rdtools.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'An open source library to support reproducible technical analysis of time series data from photovoltaic energy systems.', 'topics': '', 'organization': 'NREL', 'created': '2016/11/18, 22:17:01', 'last_update': '2021/09/17, 23:38:31', 'total_commits_last_year': 17, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/09/17, 23:38:32', 'last_released_date': '2021/09/17, 23:43:12', 'last_release_tag_name': '2.1.0', 'total_number_of_commits': 558, 'community_development_distribution_score': 0.46981627296587924, 'stargazers_count': 99, 'asset_downloads': 0, 'dominating_languages': 'Python', 'languages': 'Python', 'homepage': 'https://rdtools.readthedocs.io/', 'closed_issues': 267, 'open_issues': 31, 'closed_pullrequests': 175, 'open_pullrequests': 3, 'good_first_issue': 0, 'license': 'MIT', 'contributors': 10, 'accep

{'project_name': 'pvcaptest', 'project_namespace': 'pvcaptest', 'project_url': 'https://github.com/pvcaptest/pvcaptest.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Collection of functions and Jupyter Notebooks to partially automate running a capacity test following ASTM E2848.', 'topics': '', 'organization': 'pvcaptest', 'created': '2017/09/19, 01:33:23', 'last_update': '2021/07/26, 13:43:32', 'total_commits_last_year': 40, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/07/25, 16:51:03', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 606, 'community_development_distribution_score': 0.0018832391713747842, 'stargazers_count': 12, 'asset_downloads': 0, 'dominating_languages': 'Python', 'languages': 'Python', 'homepage': None, 'closed_issues': 39, 'open_issues': 27, 'closed_pullrequests': 15, 'open_pullrequests': 1, 'good_first_issue': 0, 'license': 'MIT', 'contributors': 2, 'accepts_donations': 'false', 'donation_platform

{'project_name': 'bifacial_radiance', 'project_namespace': 'NREL', 'project_url': 'https://github.com/NREL/bifacial_radiance.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Toolkit for working with RADIANCE for the ray-trace modeling of Bifacial Photovoltaics.', 'topics': 'radiance,bifacial,photovoltaics,renewable-energy,renewables,gui', 'organization': '', 'created': '2017/12/13, 17:56:18', 'last_update': '2021/10/25, 22:59:29', 'total_commits_last_year': 4, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/10/28, 17:17:27', 'last_released_date': '2020/05/27, 15:45:16', 'last_release_tag_name': '0.3.4', 'total_number_of_commits': 1025, 'community_development_distribution_score': 0.29161118508655126, 'stargazers_count': 50, 'asset_downloads': 0, 'dominating_languages': 'Python', 'languages': 'Python,C++,TeX,C,Makefile,Awk', 'homepage': 'https://bifacial-radiance.readthedocs.io', 'closed_issues': 280, 'open_issues': 58, 'closed_pullrequests': 95, 'open_pullreq

{'project_name': 'bifacialvf', 'project_namespace': 'NREL', 'project_url': 'https://github.com/NREL/bifacialvf.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Bifacial PV View Factor model for system performance calculation.', 'topics': '', 'organization': '', 'created': '2017/12/13, 19:23:14', 'last_update': '2021/11/08, 18:54:20', 'total_commits_last_year': 0, 'project_active_checkbox': 'false', 'last_issue_closed': '2020/06/09, 02:12:40', 'last_released_date': '2019/07/05, 20:31:42', 'last_release_tag_name': '0.1.7', 'total_number_of_commits': 92, 'community_development_distribution_score': 0.22857142857142854, 'stargazers_count': 16, 'asset_downloads': 0, 'dominating_languages': 'Python', 'languages': 'Python', 'homepage': '', 'closed_issues': 32, 'open_issues': 13, 'closed_pullrequests': 18, 'open_pullrequests': 3, 'good_first_issue': 0, 'license': 'CUSTOM', 'contributors': 3, 'accepts_donations': 'false', 'donation_platforms': None, 'code_of_conduct': 'false', 'con

...

{'project_name': 'ModularSensors', 'project_namespace': 'EnviroDIY', 'project_url': 'https://github.com/EnviroDIY/ModularSensors.git', 'rubric': 'Data and Knowledge Platforms', 'oneliner': 'This Arduino library gives environmental sensors a common interface of functions for use with Arduino-compatible dataloggers.', 'topics': 'arduino-library,environmental-sensors', 'organization': 'EnviroDIY', 'created': '2017/02/17, 19:20:33', 'last_update': '2021/11/03, 12:22:58', 'total_commits_last_year': 226, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/11/03, 12:20:13', 'last_released_date': '2021/11/03, 12:26:07', 'last_release_tag_name': 'v0.31.2', 'total_number_of_commits': 2285, 'community_development_distribution_score': 0.07984031936127745, 'stargazers_count': 57, 'asset_downloads': 66, 'dominating_languages': 'C++', 'languages': 'C++,C,Shell,PowerShell,Python', 'homepage': 'https://envirodiy.github.io/ModularSensors/', 'closed_issues': 331, 'open_issues': 56, 'closed_pull

{'organization_name': 'regen-network', 'organization_avatar': 'https://avatars.githubusercontent.com/u/32378558?v=4', 'organization_public_repos': 64, 'organization_location': None, 'organization_description': None, 'organization_public_members': 0, 'organization_repos_count': 64, 'total_organization_stars': 229}
Project Data:
{'project_name': 'Regen Ledger', 'project_namespace': 'regen-network', 'project_url': 'https://github.com/regen-network/regen-ledger.git', 'rubric': 'Data and Knowledge Platforms', 'oneliner': 'Providing a structured database of claims regarding ecological state and change of state.', 'topics': 'blockchain,ecology,sustainability,climate-change,cosmos-sdk,tendermint,agriculture', 'organization': 'regen-network', 'created': '2018/05/16, 21:10:11', 'last_update': '2021/11/08, 05:06:24', 'total_commits_last_year': 179, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/11/03, 22:19:38', 'last_released_date': '2021/10/29, 17:49:43', 'last_release_tag_name':

{'project_name': 'Awesome Spectral Indices', 'project_namespace': 'davemlz', 'project_url': 'https://github.com/davemlz/awesome-spectral-indices.git', 'rubric': 'Data and Knowledge Platforms', 'oneliner': 'A ready-to-use curated list of Spectral Indices for Remote Sensing applications.', 'topics': 'python,google,gis,raster,python3,remote-sensing,satellite-imagery,google-earth-engine,earth-engine,geographic-information-systems,javascript,awesome,landsat,sentinel,modis,landsat-8,sentinel-2,landsat-7,spectral-indices,spectral-index', 'organization': None, 'created': '2021/04/08, 00:41:25', 'last_update': '2021/11/12, 18:55:03', 'total_commits_last_year': 173, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/04/26, 14:19:41', 'last_released_date': '2021/11/08, 09:12:49', 'last_release_tag_name': '0.0.3', 'total_number_of_commits': 180, 'community_development_distribution_score': 0.2832369942196532, 'stargazers_count': 157, 'asset_downloads': 0, 'dominating_languages': 'Python'

{'project_name': 'giscoR', 'project_namespace': 'rOpenGov', 'project_url': 'https://github.com/rOpenGov/giscoR.git', 'rubric': 'Data and Knowledge Platforms', 'oneliner': 'An R API package that helps to retrieve data from Eurostat Geographic Information System of the Commission.', 'topics': 'r,gisco,eurostat-data,api-wrapper,eurostat,cran,thematic-maps,r-package,rstats,spatial,ggplot2,ropengov', 'organization': '', 'created': '2020/09/23, 19:40:44', 'last_update': '2021/11/12, 13:43:17', 'total_commits_last_year': 364, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/10/28, 19:59:51', 'last_released_date': '2021/10/06, 12:04:21', 'last_release_tag_name': 'v0.3.1', 'total_number_of_commits': 632, 'community_development_distribution_score': 0.154485049833887, 'stargazers_count': 26, 'asset_downloads': 0, 'dominating_languages': 'R', 'languages': 'R', 'homepage': 'https://ropengov.github.io/giscoR/', 'closed_issues': 42, 'open_issues': 0, 'closed_pullrequests': 35, 'open_pull

{'project_name': 'Climate Action Plans', 'project_namespace': 'mysociety', 'project_url': 'https://github.com/mysociety/caps.git', 'rubric': 'Data and Knowledge Platforms', 'oneliner': 'A simple, open database of local government climate action plan documents and emissions data.', 'topics': 'climate,climate-change,local-government,citizen-participation,mapit,docker,vagrant', 'organization': 'mysociety', 'created': '2020/08/21, 20:17:51', 'last_update': '2021/11/10, 14:53:44', 'total_commits_last_year': 122, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/11/08, 11:01:18', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 282, 'community_development_distribution_score': 0.5893536121673004, 'stargazers_count': 4, 'asset_downloads': 0, 'dominating_languages': 'Python', 'languages': 'Python,HTML,SCSS,JavaScript,Shell,Dockerfile', 'homepage': 'https://data.climateemergency.uk', 'closed_issues': 115, 'open_issues': 47, 'closed_pullrequests': 71, 